# Setup Library

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd.variable import Variable
from torchvision import transforms

import os

In [11]:
# Tentukan device yang akan digunakan (GPU jika tersedia, jika tidak, gunakan CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Setup Data

Datset = https://www.kaggle.com/datasets/hojjatk/mnist-dataset

In [12]:
import os

mainPath = "/content/drive/MyDrive/Colab Notebooks/1. BISA AI - [Studi Independen]/Computer Vision - Open CV/Enroll Generative Adversarial Networks dengan Pytorch"
dataPath = "/content/drive/MyDrive/Colab Notebooks/7. Dataset Bisa AI/MNIST_Digits"

In [ ]:
# # Set-up Env Variable for kaggle Config
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = mainPath

# # Tentukan path untuk menyimpan dataset
# path_to_save = dataPath

# # Buat direktori jika belum ada
# os.makedirs(path_to_save, exist_ok=True)

# # Unduh dataset menggunakan perintah kaggle
# !kaggle datasets download -d hojjatk/mnist-dataset -p '{path_to_save}'

In [ ]:
# # Ekstrak dataset
# !unzip '{path_to_save}/mnist-dataset.zip' -d '{path_to_save}'

# # Hapus file zip jika diperlukan
# os.remove(f'{path_to_save}/mnist-dataset.zip')

In [ ]:
import tensorflow as tf

# Load MNIST dataset
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0

# Reshape images to (batch_size, height, width, channels)
train_images = train_images.reshape(-1, 28, 28, 1)

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(train_images)

# Shuffle and batch the dataset
batch_size = 128
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

11490434/11490434 [==============================] - 1s 0us/step


In [18]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Define the discriminator network
def discriminator_net():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define the generator network
def generator_net():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(128,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(28*28, activation='tanh'),
        tf.keras.layers.Reshape((28, 28, 1))
    ])
    return model

# Create instances of discriminator and generator
discriminator = discriminator_net()
generator = generator_net()

# Define loss function and optimizers
loss = tf.keras.losses.BinaryCrossentropy()
d_optimizer = tf.keras.optimizers.Adam(0.0002)
g_optimizer = tf.keras.optimizers.Adam(0.0002)

In [19]:
# Print model summaries
discriminator.summary()
generator.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_32 (Dense)            (None, 1024)              803840    
                                                                 
 dropout_12 (Dropout)        (None, 1024)              0         
                                                                 
 dense_33 (Dense)            (None, 512)               524800    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                                 
 dense_34 (Dense)            (None, 256)               131328    
                                                                 
 dropout_14 (Dropout)        (None, 256)              

In [20]:
# Training loop
EPOCHS = 200

for epoch in range(EPOCHS):
    for batch in train_dataset:
        # Flatten the batch of images
        real_data = tf.reshape(batch[0], (-1, 28, 28, 1))  # Assuming batch[0] contains the images

        # Train Discriminator
        with tf.GradientTape() as tape:
            # Train on real data
            pred_real = discriminator(real_data)
            error_real = loss(tf.ones_like(pred_real), pred_real)

            # Train on fake data
            noise_tensor = tf.random.normal((batch.shape[0], 128))
            fake_data = generator(noise_tensor)
            pred_fake = discriminator(fake_data)
            error_fake = loss(tf.zeros_like(pred_fake), pred_fake)

            # Total discriminator loss
            d_error = error_real + error_fake

        # Compute gradients and update discriminator weights
        d_gradients = tape.gradient(d_error, discriminator.trainable_variables)
        d_optimizer.apply_gradients(zip(d_gradients, discriminator.trainable_variables))

        # Train Generator
        with tf.GradientTape() as tape:
            # Generate fake data
            noise_tensor = tf.random.normal((batch.shape[0], 128))
            fake_data = generator(noise_tensor)

            # Get Discriminator's prediction on fake data
            pred = discriminator(fake_data)

            # Generator loss
            g_error = loss(tf.ones_like(pred), pred)

        # Compute gradients and update generator weights
        g_gradients = tape.gradient(g_error, generator.trainable_variables)
        g_optimizer.apply_gradients(zip(g_gradients, generator.trainable_variables))

    # Log and display progress after each epoch
    print(
        f"Epoch [{epoch+1}/{EPOCHS}], D Loss: {d_error.numpy():.4f}, G Loss: {g_error.numpy():.4f}"
    )

    # Display generated images after each epoch
    test_noise = tf.random.normal((16, 128))
    test_images = generator(test_noise)
    fig, axs = plt.subplots(4, 4, figsize=(8, 8))
    for i, ax in enumerate(axs.flatten()):
        ax.imshow(tf.squeeze(test_images[i]), cmap='gray')
        ax.axis('off')
    plt.show()

Output hidden; open in https://colab.research.google.com to view.